# Overview
The purpose of this project is to perform portfolio optimization based on forward-looking indicators. This involves using an ML model to predict returns for stocks and bonds.

In [ ]:
# Imports
import datetime
import torch
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Load the Philadelphia Fed survey of professional forecasters (SPF)data

# GDP Growth
PHILLY_FED_GROWTH_DATA = pd.read_csv('phillyfedgrowthrate.csv')
# Inflation
PHILLY_FED_INFLATION_DATA = pd.read_csv('phillyfedinflationrate.csv')
# Interest
PHILLY_FED_INTEREST_DATA = pd.read_csv('phillyfedinterestrate.csv')
# Risk Premium (Damodaran)
DAMODARAN_RISK_PREMIUM_DATA = pd.read_csv('damodaranriskpremium.csv', skiprows=6)

# Helper functions
    
def get_expected_gdp_growth(date):
    """
    Get GDP growth predictions from Philadelphia Federal Reserve 
    Survey of Professional Forecasters using DRGDP3 data.
    
    Args:
        date: datetime object for which we want the GDP growth prediction
    
    Returns:
        Expected GDP growth rate as a float (e.g., 0.025 for 2.5%)
    """
    try:
        # Parse datetime to year and quarter
        year = date.year
        quarter = (date.month - 1) // 3 + 1
        
        # Find the row with the target date
        mask = (PHILLY_FED_GROWTH_DATA['YEAR'] == year) & (PHILLY_FED_GROWTH_DATA['QUARTER'] == quarter)
        data_row = PHILLY_FED_GROWTH_DATA[mask]
        
        if not data_row.empty:
            # Get DRGDP3 value from the target quarter
            drgdp3_value = data_row['DRGDP3'].iloc[0]
            
            # Handle missing data
            if pd.isna(drgdp3_value) or drgdp3_value == '#N/A':
                # Try DRGDP2 as backup
                drgdp2_value = data_row['DRGDP2'].iloc[0]
                if not pd.isna(drgdp2_value) and drgdp2_value != '#N/A':
                    return float(drgdp2_value) / 100.0
                
                # Final fallback
                return 0.025  # 2.5% default
            else:
                return float(drgdp3_value) / 100.0  # Convert percentage to decimal
        else:
            # No data found for the exact quarter, return default
            print(f"Warning: GDP growth data for quarter {quarter} {year} not found. Using default value of 2.5%.")
            return 0.025  # 2.5% default
            
    except Exception as e:
        print(f"Warning: GDP growth data for quarter {quarter} {year} not properly loaded. Using default value of 2.5%.")
        return 0.025  # 2.5% safe default


def get_expected_inflation(date):
    """
    Get inflation predictions from Philadelphia Federal Reserve 
    Survey of Professional Forecasters using INFPGDP1YR data 
    (1 year GDP deflator).
    
    Args:
        date: datetime object for which we want the inflation prediction
    
    Returns:
        Expected inflation rate as a float (e.g., 0.025 for 2.5%)
    """
    try:
        # Parse datetime to year and quarter
        year = date.year
        quarter = (date.month - 1) // 3 + 1
        
        # Find the row with the target date
        mask = (PHILLY_FED_INFLATION_DATA['YEAR'] == year) & (PHILLY_FED_INFLATION_DATA['QUARTER'] == quarter)
        data_row = PHILLY_FED_INFLATION_DATA[mask]

        if not data_row.empty:
            # Get INFPGDP1YR value from the target quarter
            infpgdp1yr_value = data_row['INFPGDP1YR'].iloc[0]
            
            # Handle missing data
            if pd.isna(infpgdp1yr_value) or infpgdp1yr_value == '#N/A':
                print(f"Warning: Inflation data for quarter {quarter} {year} not found. Trying next quarters.")
                # Try to find data for the next quarter(s) as fallback
                for next_quarters in range(1, 5):  # Try up to 4 quarters ahead
                    next_quarter = quarter + next_quarters
                    next_year = year
                    
                    # Handle year rollover
                    if next_quarter > 4:
                        next_quarter = next_quarter - 4
                        next_year = year + 1

                    next_mask = (PHILLY_FED_INFLATION_DATA['YEAR'] == next_year) & (PHILLY_FED_INFLATION_DATA['QUARTER'] == next_quarter)
                    next_row = PHILLY_FED_INFLATION_DATA[next_mask]

                    if not next_row.empty:
                        next_value = next_row['INFPGDP1YR'].iloc[0]
                        if not pd.isna(next_value) and next_value != '#N/A':
                            return float(next_value) / 100.0
                
                # Final fallback
                print(f"Warning: Inflation data for quarter {quarter} {year} not found. Using default value of 3%.")
                return 0.03  # 3% default
            else:
                return float(infpgdp1yr_value) / 100.0  # Convert percentage to decimal
        else:
            # No data found for the exact quarter, return default
            print(f"Warning: Inflation data for quarter {quarter} {year} not found. Using default value of 3%.")
            return 0.03  # 3% default
            
    except Exception as e:
        print(f"Error: {e}. Using default value of 3%.")
        return 0.03  # 3% safe default


def get_expected_interest_rate(date):
    """
    Get interest rate predictions from Philadelphia Federal Reserve 
    Survey of Professional Forecasters using TBILL1 data 
    (3-month Treasury bill rate forecast).
    
    Args:
        date: datetime object for which we want the interest rate prediction
    
    Returns:
        Expected interest rate as a float (e.g., 0.025 for 2.5%)
    """
    try:
        # Parse datetime to year and quarter
        year = date.year
        quarter = (date.month - 1) // 3 + 1
        
        # Find the row with the target date
        mask = (PHILLY_FED_INTEREST_DATA['YEAR'] == year) & (PHILLY_FED_INTEREST_DATA['QUARTER'] == quarter)
        data_row = PHILLY_FED_INTEREST_DATA[mask]

        if not data_row.empty:
            # Get TBILL1 value from the target quarter
            tbill1_value = data_row['TBILL1'].iloc[0]
            
            # Handle missing data
            if pd.isna(tbill1_value) or tbill1_value == '#N/A':
                # Try TBILL2 as backup
                tbill2_value = data_row['TBILL2'].iloc[0]
                if not pd.isna(tbill2_value) and tbill2_value != '#N/A':
                    return float(tbill2_value) / 100.0
                
                # Final fallback
                print(f"Warning: Interest rate data for quarter {quarter} {year} not found. Using default value of 1%.")
                return 0.01  # 1% default
            else:
                return float(tbill1_value) / 100.0  # Convert percentage to decimal
        else:
            # No data found for the exact quarter, return default
            print(f"Warning: Interest rate data for quarter {quarter} {year} not found. Using default value of 1%.")
            return 0.01  # 1% default
            
    except Exception as e:
        print(f"Error: {e}. Using default value of 1%.")
        return 0.01  # 1% safe default

def get_risk_premium(date):
    """
    Get equity risk premium from Damodaran's historical data using 
    Implied ERP (FCFE) - Free Cash Flow to Equity model.
    
    Args:
        date: datetime object for which we want the risk premium
    
    Returns:
        Expected risk premium as a float (e.g., 0.05 for 5%)
    """
    try:
        # Parse datetime to year
        year = date.year
        
        # Find the row with the target year
        # Clean the data first - remove rows with non-numeric years
        clean_data = DAMODARAN_RISK_PREMIUM_DATA[
            pd.to_numeric(DAMODARAN_RISK_PREMIUM_DATA['Year'], errors='coerce').notna()
        ].copy()
        
        # Convert Year column to int
        clean_data['Year'] = clean_data['Year'].astype(int)
        
        # Find exact year match
        year_data = clean_data[clean_data['Year'] == year]
        
        if not year_data.empty:
            # Get Implied ERP (FCFE) value
            erp_fcfe_value = year_data['Implied ERP (FCFE)'].iloc[0]
            
            # Handle missing data or string formatting
            if pd.isna(erp_fcfe_value) or erp_fcfe_value == '' or erp_fcfe_value == '#N/A':
                # Try to use most recent available data
                recent_data = clean_data[clean_data['Year'] < year].sort_values('Year', ascending=False)
                
                if not recent_data.empty:
                    for _, row in recent_data.head(5).iterrows():  # Check last 5 years
                        recent_erp = row['Implied ERP (FCFE)']
                        if not pd.isna(recent_erp) and recent_erp != '' and recent_erp != '#N/A':
                            # Convert percentage string to float
                            if isinstance(recent_erp, str) and '%' in recent_erp:
                                return float(recent_erp.replace('%', '')) / 100.0
                            else:
                                return float(recent_erp) / 100.0
                
                # Final fallback
                print(f"Warning: Risk premium data for year {year} not found. Using default value of 5%.")
                return 0.05  # 5% default
            else:
                # Convert percentage string to float
                if isinstance(erp_fcfe_value, str) and '%' in erp_fcfe_value:
                    return float(erp_fcfe_value.replace('%', '')) / 100.0
                else:
                    return float(erp_fcfe_value) / 100.0
        else:
            # No exact year match, use most recent available data before the target year
            historical_data = clean_data[clean_data['Year'] <= year].sort_values('Year', ascending=False)
            
            if not historical_data.empty:
                recent_erp = historical_data['Implied ERP (FCFE)'].iloc[0]
                if not pd.isna(recent_erp) and recent_erp != '' and recent_erp != '#N/A':
                    # Convert percentage string to float
                    if isinstance(recent_erp, str) and '%' in recent_erp:
                        return float(recent_erp.replace('%', '')) / 100.0
                    else:
                        return float(recent_erp) / 100.0
            
            # Final fallback
            print(f"Warning: Risk premium data for year {year} not found. Using default value of 5%.")
            return 0.05  # 5% default
            
    except Exception as e:
        print(f"Error: {e}. Using default risk premium of 5%.")
        return 0.05  # 5% safe default